# Introduction + Sources, Resources, and Pipelines

## 1. Introduction

### What is dlt?

[dlt](https://dlthub.com/) is a Python data ingestion framework enabling data engineers to define connectors and pipelines as code. It offers a rich set of features for building best-practice pipelines and supports both built-in and custom connectors built with regular Python code.

For more information, see the [official docs](https://dlthub.com/docs/intro#what-is-dlt).

### How does it work?

dlt ingests data in three stages: extract, normalize, and load. The extract stage downloads source data to disk. The normalize stage applies light transformations to the data, such as column renaming or datetime parsing. The load stage loads the data into the destination system.

![how does dlt work diagram](https://dlthub.com/docs/assets/images/dlt-onepager-c61255330e30060ca8f2fa6d7b73b600.png)
Credit: [dlt documentation](https://dlthub.com/docs/reference/explainers/how-dlt-works)

## 2. Hello, dlt

Let's jump in and see dlt in action!



In [3]:
import dlt


# Sample data.
people = [
    {"id": "1", "name": "Warren Buffet", "country": "USA"},
    {"id": "2", "name": "Jack Ma", "country": "China"},
    {"id": "3", "name": "Rafal Brzoska", "country": "Poland"},
]

# Set pipeline name, destination, and dataset name.
pipeline = dlt.pipeline(
    pipeline_name="dummy_source_to_duckdb",
    destination="duckdb",
    dataset_name="mydata",
)

load_info = pipeline.run(people, table_name="person")

print(load_info)

Pipeline dummy_source_to_duckdb load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata
The duckdb destination used duckdb:////workspaces/edp-workshop/dummy_source_to_duckdb.duckdb location to store data
Load package 1742325689.3566227 is LOADED and contains no failed jobs


Let's see what was loaded into the database:

In [14]:
!echo "select table_catalog, table_schema, table_name from information_schema.tables;" | duckdb ../dummy_source_to_duckdb.duckdb

┌────────────────────────┬──────────────┬─────────────────────┐
│     table_catalog      │ table_schema │     table_name      │
│        varchar         │   varchar    │       varchar       │
├────────────────────────┼──────────────┼─────────────────────┤
│ dummy_source_to_duckdb │ mydata       │ person              │
│ dummy_source_to_duckdb │ mydata       │ _dlt_loads          │
│ dummy_source_to_duckdb │ mydata       │ _dlt_pipeline_state │
│ dummy_source_to_duckdb │ mydata       │ _dlt_version        │
└────────────────────────┴──────────────┴─────────────────────┘


In [10]:
!echo "select * from mydata.person;" | duckdb ../dummy_source_to_duckdb.duckdb

┌─────────┬───────────────┬─────────┬────────────────────┬────────────────┐
│   id    │     name      │ country │    _dlt_load_id    │    _dlt_id     │
│ varchar │    varchar    │ varchar │      varchar       │    varchar     │
├─────────┼───────────────┼─────────┼────────────────────┼────────────────┤
│ 1       │ Warren Buffet │ USA     │ 1742325689.3566227 │ qqJPygfKATrxhg │
│ 2       │ Jack Ma       │ China   │ 1742325689.3566227 │ 7BtQ7b/W0BYtCw │
│ 3       │ Rafal Brzoska │ Poland  │ 1742325689.3566227 │ GgzsEwOEnPesYQ │
└─────────┴───────────────┴─────────┴────────────────────┴────────────────┘


In the next sections, we will learn more about the features and ways of working with dlt as we build gradually more complex pipelines.

## 3. Sources, resources, and pipelines

In this section, we will:

- learn about the three key dlt concepts
- configure a source and two resources
- briefly showcase destinations

### 3.1 Resources

Resources represent the data that flows through a dlt pipeline. They allow us to use various dlt funcionalities such as incremental loading and specify some of the ELT configuration, which we would be unable to do if we worked with raw data.

In the script below, we use `dlt.resource()` in order to specify the name of the target table when defining the resource, rather than at pipeline runtime (`pipeline.run()`).

In [10]:
import dlt

# We now describe the data source as a dlt resource rather than a Python list.
@dlt.resource(table_name="person")
def people():
    yield [
        {"id": "1", "name": "Warren Buffet", "country": "USA"},
        {"id": "2", "name": "Jack Ma", "country": "China"},
        {"id": "3", "name": "Rafal Brzoska", "country": "Poland"},
    ]

pipeline = dlt.pipeline(
    pipeline_name="dummy_source_to_duckdb",
    destination="duckdb",
    dataset_name="mydata",
)

load_info = pipeline.run(people)

print(load_info)

Pipeline dummy_source_to_duckdb load step completed in 0.17 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata
The duckdb destination used duckdb:////workspaces/edp-workshop/dummy_source_to_duckdb.duckdb location to store data
Load package 1742311418.0469034 is LOADED and contains no failed jobs


### 3.2 Sources

Sources are groups of resources. They allow us to define the source of the data and the resources that will be loaded from that source. For example, a source could be an SQL database, while a resource would be a table in that database.

dlt offers several built-in standard sources such as databases, REST APIs, or cloud storage. We can also define custom sources by using the `dlt.source()` decorator.

In the script below, we define a custom source with two resources.

In [1]:
import dlt


@dlt.source
def dummy_data():
    @dlt.resource(table_name="person")
    def people():
        yield [
            {"id": "1", "name": "Warren Buffet", "country": "USA"},
            {"id": "2", "name": "Jack Ma", "country": "China"},
            {"id": "3", "name": "Rafal Brzoska", "country": "Poland"},
        ]

    @dlt.resource(table_name="country")
    def countries():
        yield [
            {"id": "1", "name": "USA", "population": 331449281},
            {"id": "2", "name": "China", "population": 1444216107},
            {"id": "3", "name": "Poland", "population": 37846611},
        ]

    # NOTE: We need to return the resources here.
    return people, countries


pipeline = dlt.pipeline(
    pipeline_name="dummy_source_to_duckdb",
    destination="duckdb",
    dataset_name="mydata",
)

if __name__ == "__main__":
    # NOTE: We still provide resources as the data, since `dummy_data()` returns
    # the two resources.
    load_info = pipeline.run(data=dummy_data())

print(load_info)

Pipeline dummy_source_to_duckdb load step completed in 0.19 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata
The duckdb destination used duckdb:////workspaces/edp-workshop/dummy_source_to_duckdb.duckdb location to store data
Load package 1742313799.8798592 is LOADED and contains no failed jobs


##### A more real-life example

Just to show you how easy this is with any built-in dlt source as well, let's sidestep and quickly load some data from a production MySQL database:

In [ ]:
%%capture

# Install required dependencies for the MySQL connector.
!uv add dlt --extra sql_database
!uv add pymysql

NOTE: executing this might take a minute.

In [ ]:
from dlt.sources.sql_database import sql_database

# NOTE: without .with_resources(), we'd be replicating the entire database, which is
# too resource-intensive for this tutorial.
source = sql_database(
    "mysql+pymysql://rfamro@mysql-rfam-public.ebi.ac.uk:4497/Rfam"
).with_resources("family", "genome")

pipeline = dlt.pipeline(
    pipeline_name="sql_database_example",
    destination="duckdb",
    dataset_name="sql_data",
)

load_info = pipeline.run(source)

print(load_info)

Pipeline sql_database_example load step completed in 6.43 seconds
1 load package(s) were loaded to destination duckdb and into dataset sql_data
The duckdb destination used duckdb:////workspaces/edp-workshop/sql_database_example.duckdb location to store data
Load package 1742329216.0123036 is LOADED and contains no failed jobs


In [12]:
!echo "select * from sql_data.family limit 3;" | duckdb ../sql_database_example.duckdb

┌──────────┬───────────┬───────────┬───┬──────────────────────┬──────────────────────┬────────────────────┬────────────────┐
│ rfam_acc │  rfam_id  │ auto_wiki │ … │       created        │       updated        │    _dlt_load_id    │    _dlt_id     │
│ varchar  │  varchar  │   int64   │   │ timestamp with tim…  │ timestamp with tim…  │      varchar       │    varchar     │
├──────────┼───────────┼───────────┼───┼──────────────────────┼──────────────────────┼────────────────────┼────────────────┤
│ RF00001  │ 5S_rRNA   │      1302 │ … │ 2013-10-03 20:41:4…  │ 2024-09-09 21:15:1…  │ 1742329216.0123036 │ xlL35qI6gfeG+A │
│ RF00002  │ 5_8S_rRNA │      1303 │ … │ 2013-10-03 20:47:0…  │ 2024-09-10 04:51:1…  │ 1742329216.0123036 │ kjStHhrqk+sMBA │
│ RF00003  │ U1        │      1304 │ … │ 2013-10-03 20:57:1…  │ 2024-09-09 21:15:1…  │ 1742329216.0123036 │ lzsAHzptTBCjrg │
├──────────┴───────────┴───────────┴───┴──────────────────────┴──────────────────────┴────────────────────┴────────────────┤


That's it! Just like that, we loaded real-life data from a MySQL database into our local DuckDB instance*.

*Note that this is a public database and so we didn't have to specify any credentials. We'll learn about those later in the workshop.

### 3.3 Pipelines

In `dlt`, a pipeline describes the flow of data from resource(s) to a destination. Each pipeline loads resources to a single destination.

Pipelines can be reused to ingest different resources each run. For example, we can have one “Postgres to S3” pipeline, but ingest each Postgres table separately due to different scheduling or configuration needs.

A pipeline definition contains pipeline- or pipeline run-specific destination configuration, as well as settings for the load phase of the ingestion. Under the hood, a pipeline run (`pipeline.run()`) executes each pipeline step: extract (`pipeline.extract()`), normalize (`pipeline.normalize()`), and load (`pipeline.load()`).

Let's use this knowledge to better control our destination configuration - in this case, we'll control the name of the DuckDB database where the data will be loaded.

In [16]:
import dlt


@dlt.source
def dummy_data():
    @dlt.resource(table_name="person")
    def people():
        yield [
            {"id": "1", "name": "Warren Buffet", "country": "USA"},
            {"id": "2", "name": "Jack Ma", "country": "China"},
            {"id": "3", "name": "Rafal Brzoska", "country": "Poland"},
        ]

    @dlt.resource(table_name="country")
    def countries():
        yield [
            {"id": "1", "name": "USA", "population": 331449281},
            {"id": "2", "name": "China", "population": 1444216107},
            {"id": "3", "name": "Poland", "population": 37846611},
        ]

    return people, countries


pipeline = dlt.pipeline(
    pipeline_name="dummy_source_to_duckdb",
    destination=dlt.destinations.duckdb("db.duckdb"), # NOTE: we renamed the db here.
    dataset_name="mydata",
)

if __name__ == "__main__":
    load_info = pipeline.run(data=dummy_data())

print(load_info)

Pipeline dummy_source_to_duckdb load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata
The duckdb destination used duckdb:////workspaces/edp-workshop/db.duckdb location to store data
Load package 1742376576.7059135 is LOADED and contains no failed jobs


#### Inspecting data in the destination

`dlt` offers two main built-in ways to inspect the data in the destination, SQL client and datasets.

##### SQL client

In [19]:
%%capture

# Install required dependencies for the SQL client.
!uv add pandas

In [ ]:
with pipeline.sql_client() as client:
    with client.execute_query("SELECT * FROM mydata.person") as cursor:
        data = cursor.df()

data

,id,name,country,_dlt_load_id,_dlt_id
0,1,Warren Buffet,USA,1742376576.7059135,VErrKTHuqp5UeA
1,2,Jack Ma,China,1742376576.7059135,AexicFW7y3LwOQ
2,3,Rafal Brzoska,Poland,1742376576.7059135,cNUGMrmkTi2Pjw


##### Dataset

In [23]:
dataset = pipeline.dataset(dataset_type="default")
dataset.country.df()

,id,name,population,_dlt_load_id,_dlt_id
0,1,USA,331449281,1742376576.7059135,RqI7Ai7vZ2GFow
1,2,China,1444216107,1742376576.7059135,IINegGQJ+hc4Og
2,3,Poland,37846611,1742376576.7059135,I1+swwZUv2YHww


## 4. Exercise

Let's now use all of the knowledge we've gained to build a pipeline that loads data from two CSV files into a DuckDB database:

- define a source, `csvs`, that will contain CSV file resources
- use provided two util functions to define two resources, `iris` and `wine`
- define a pipeline that will load the resources into a `bronze` schema in a `csvs.duckdb` DuckdDB database

In [ ]:
# Your solution here.
import dlt
import pandas as pd


def read_iris():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
    col_names = ['Sepal_Length','Sepal_Width','Petal_Length','Petal_Width','Class']
    return pd.read_csv(url, names=col_names).to_dict(orient="records")


def read_wine():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"
    col_names = [
        "fixed_acidity",
        "volatile_acidity",
        "citric_acid",
        "residual_sugar",
        "chlorides",
        "free_sulfur_dioxide",
        "total_sulfur_dioxide",
        "density",
        "pH",
        "sulphates",
        "alcohol",
        "quality",
        "color",
    ]
    return pd.read_csv(url, names=col_names).to_dict(orient="records")

# Define the resource.
...

# Define the pipeline.
...

if __name__ == "__main__":
    # Run the pipeline and print load info.
    ...

Ellipsis

#### Solution

Do not execute or peek!

In [79]:
import dlt
import pandas as pd


def read_iris():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
    col_names = ['Sepal_Length','Sepal_Width','Petal_Length','Petal_Width','Class']
    return pd.read_csv(url, names=col_names).to_dict(orient="records")


def read_wine():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"
    col_names = [
        "fixed_acidity",
        "volatile_acidity",
        "citric_acid",
        "residual_sugar",
        "chlorides",
        "free_sulfur_dioxide",
        "total_sulfur_dioxide",
        "density",
        "pH",
        "sulphates",
        "alcohol",
        "quality",
        "color",
    ]
    return pd.read_csv(url, names=col_names).to_dict(orient="records")


# NOTE: By default, dlt uses the name of the resource as the name of the destination
# table.
@dlt.source
def csvs():
    @dlt.resource()
    def iris():
        yield read_iris()

    @dlt.resource()
    def wine():
        yield read_wine()

    return iris, wine


pipeline = dlt.pipeline(
    pipeline_name="dummy_source_to_duckdb",
    destination=dlt.destinations.duckdb("csvs.duckdb"),
    dataset_name="bronze",
)

if __name__ == "__main__":
    load_info = pipeline.run(csvs())

print(load_info)

Pipeline dummy_source_to_duckdb load step completed in 0.27 seconds
1 load package(s) were loaded to destination duckdb and into dataset bronze
The duckdb destination used duckdb:////workspaces/edp-workshop/csvs.duckdb location to store data
Load package 1742380861.9593136 is LOADED and contains no failed jobs


If everything went well, we should be able to query the data now:

In [80]:
!echo "select * from bronze.iris limit 3;" | duckdb ../csvs.duckdb

┌──────────────┬─────────────┬──────────────┬─────────────┬─────────────┬────────────────────┬────────────────┐
│ sepal_length │ sepal_width │ petal_length │ petal_width │    class    │    _dlt_load_id    │    _dlt_id     │
│    double    │   double    │    double    │   double    │   varchar   │      varchar       │    varchar     │
├──────────────┼─────────────┼──────────────┼─────────────┼─────────────┼────────────────────┼────────────────┤
│          5.1 │         3.5 │          1.4 │         0.2 │ Iris-setosa │ 1742380861.9593136 │ huuC9VWEUVsMyA │
│          4.9 │         3.0 │          1.4 │         0.2 │ Iris-setosa │ 1742380861.9593136 │ k+Psl0l+gB0qLA │
│          4.7 │         3.2 │          1.3 │         0.2 │ Iris-setosa │ 1742380861.9593136 │ J4zDkQRIQ9xSsA │
└──────────────┴─────────────┴──────────────┴─────────────┴─────────────┴────────────────────┴────────────────┘


In [81]:
!echo "select * from bronze.wine limit 3;" | duckdb ../csvs.duckdb

┌───────────────┬──────────────────┬─────────────┬────────────────┬───┬─────────┬─────────┬───────┬────────────────────┬────────────────┐
│ fixed_acidity │ volatile_acidity │ citric_acid │ residual_sugar │ … │ alcohol │ quality │ color │    _dlt_load_id    │    _dlt_id     │
│    double     │      double      │   double    │     double     │   │ double  │ double  │ int64 │      varchar       │    varchar     │
├───────────────┼──────────────────┼─────────────┼────────────────┼───┼─────────┼─────────┼───────┼────────────────────┼────────────────┤
│         14.23 │             1.71 │        2.43 │           15.6 │ … │    1.04 │    3.92 │  1065 │ 1742380861.9593136 │ eGnmNq4F9t/oaQ │
│          13.2 │             1.78 │        2.14 │           11.2 │ … │    1.05 │     3.4 │  1050 │ 1742380861.9593136 │ Bnqm7wzW+cOALw │
│         13.16 │             2.36 │        2.67 │           18.6 │ … │    1.03 │    3.17 │  1185 │ 1742380861.9593136 │ MQ0DPPbpjRXWYA │
├───────────────┴─────────────────

## 5. Summary

In this lesson, we've:

- learned and got a feel for what dlt is
- learned about its fundamental concepts: sources, resources, and pipelines
- loaded some fake and actual data from various sources (a Python object, MySQL database, CSV files) into a local DuckDB database
